In [1]:
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/drive')

!pip install transformers==4.3.3
!pip install stop_words
!pip install symspellpy
!pip install language_detector 
!pip install cached_property
!pip install sentencepiece
!pip install config
!pip install umap
!pip install sentence-transformers==0.4.1
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

import tensorflow, sentence_transformers, transformers
tensorflow.__version__, sentence_transformers.__version__, transformers.__version__

root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"
import sys, os
import config

config.root_path = os.path.abspath(root_path)
sys.path.insert(0, config.root_path)

import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from src.encoders.context_encoder_ldabert_2 import ContextEncoderComplex, ContextEncoderConv
from src.dataset.ldabert_2 import LDABERT2Dataset

from tensorflow.python import keras
import toml
import json
import pandas as pd
import numpy as np
from utils.experiments import get_experiments_json, get_experiments, save_results

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 6.3 MB/s 
     |████████████████████████████████| 880 kB 51.0 MB/s 
     |████████████████████████████████| 3.3 MB 36.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=03433b496aac5a53a78e3081e2004ba6f0b2e313da53f0eb45f9972b404a7922
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=29f72750039e5ecee14ff38c79b2bca7b8464a5a74cea1f1e73d0f5231c40daf
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words
Looking

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Downloading:   0%|          | 0.00/630 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/409 [00:00<?, ?B/s]

In [27]:
print("initializing dataset...")

pct_data = 1
dataset_type = "clinical"
max_sentence_length = 128
augment_pct = 1

dataset = LDABERT2Dataset(dataset_type=dataset_type,
                          pct_data=pct_data,
                          max_seq_length=max_sentence_length,
                          max_segment_length=5,
                          augment_pct=augment_pct,
                          split="train",
                          artificial_segments=True)


sentences, tokenized_sentences, labels = dataset.process()

initializing dataset...
artificial segments True
artificial segments True


In [28]:
dataset.labels[:15]

[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]

In [11]:
vectors_filename = '{}_{}_artificial-segments-{}.pkl'.format(dataset.pct_data, dataset.augment_pct, dataset.artificial_segments)

saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.get_saved_vectors("train", dataset.dataset_type, vectors_filename)

if len(saved_vectors) == 0:
    saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.create_vectors("train", dataset.dataset_type, vectors_filename)

# left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)
# lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors)
print(saved_labels)

# get class weight
neg, pos = np.bincount(np.array(saved_labels).flatten())
initial_bias = np.log([pos/neg])

total=len(saved_labels)
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0
class_weight = {0: weight_for_0, 1: weight_for_1}
print("class weight", class_weight)

something went wrong [Errno 2] No such file or directory: '/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/data/lda_bert_2/generated_vectors/train/clinical/1_1_artificial-segments-True.pkl'
root path /content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2
Preprocessing raw texts ...
sentences length 39968
Preprocessing raw texts. Done!
lda sentences length 39968
Getting vector representations for LDA ...
Getting vector representations for LDA. Done!
saving vectors... 39968 39968 39968
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [30]:
pct_data = 1
dataset_types = ["clinical", "fiction", "wiki"]
max_sentence_length = 128
augment_pct = 1

for dataset_type in dataset_types:
  dataset = LDABERT2Dataset(dataset_type=dataset_type,
                            pct_data=pct_data,
                            max_seq_length=max_sentence_length,
                            max_segment_length=5,
                            augment_pct=augment_pct,
                            split="train",
                            artificial_segments=True)
  
  sentences, tokenized_sentences, labels = dataset.process()

  # as = artificial segment (chop off segment at certain length)
  # msl = max segment length (when used with as)
  vectors_filename = '{}_{}_as-{}_msl-{}.pkl'.format(dataset.pct_data, dataset.augment_pct, dataset.artificial_segments, dataset.max_seq_length)

  saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.get_saved_vectors("train", dataset.dataset_type, vectors_filename)

  if len(saved_vectors) == 0:
      saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.create_vectors("train", dataset.dataset_type, vectors_filename)

  # left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)
  # lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors)
  print(saved_labels)

  # get class weight
  neg, pos = np.bincount(np.array(saved_labels).flatten())
  initial_bias = np.log([pos/neg])

  total=len(saved_labels)
  weight_for_0 = (1 / neg)*(total)/2.0 
  weight_for_1 = (1 / pos)*(total)/2.0
  class_weight = {0: weight_for_0, 1: weight_for_1}
  print("class weight", class_weight)

artificial segments True
artificial segments True
something went wrong [Errno 2] No such file or directory: '/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/data/lda_bert_2/generated_vectors/train/clinical/1_1_as-True_msl-128.pkl'
root path /content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2
Preprocessing raw texts ...
sentences length 44979
Preprocessing raw texts. Done!
lda sentences length 44979
Getting vector representations for LDA ...
Getting vector representations for LDA. Done!
saving vectors... 44979 44979 44979
[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 